In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip3 install transformers

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split

## Preprocessing

In [ ]:
PATH = "/content/drive/MyDrive/Fairness/data/MBIC.xlsx"
df = pd.read_excel(PATH)
df.head()

In [ ]:
df.rename(columns={'sentence': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df.head()

In [ ]:
# binarize classification problem
df = df[df['Label_bias']!='No agreement']
df = df[df['Label_bias'].isna()==False]
df.replace(to_replace='Biased', value=1, inplace=True)
df.replace(to_replace='Non-biased', value=0, inplace=True)


### Train val split

In [ ]:
X = df['sentence']  
y = df['Label_bias']  



X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


#### Preprocessing the dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
X_train_encoded = tokenizer(
    list(X_train.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_val_encoded = tokenizer(
    list(X_val.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_test_encoded = tokenizer(
    list(X_test),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [ ]:
class Dataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = Dataset(X_train_encoded, y_train.values)
val_dataset = Dataset(X_val_encoded, y_val.values)
test_dataset = Dataset(X_test_encoded, y_test.values)
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## Training

In [ ]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
import evaluate
import numpy as np
import os
os.environ['HF_MLFLOW_LOG_ARTIFACTS'] = "1" 

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
!pip install accelerate>=0.21.0 -U

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Fairness/Bias detection/models/bert/',         
    num_train_epochs=10,              
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,   
    warmup_steps=500,                
    weight_decay=0.01,              
    logging_dir='./logs',            
    logging_steps=400,
    evaluation_strategy='steps',
    eval_steps=400,
    load_best_model_at_end=True,
    save_total_limit=3,
    save_steps=400

)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,             
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

### Calculate performance

In [ ]:
from tqdm import tqdm
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

tp = 0.0  
fp = 0.0 
tn = 0.0  
fn = 0.0  
with torch.no_grad():
    for data in tqdm(test_loader):
        input_ids, labels = data['input_ids'].to(device), data['labels'].to(device)
        out = torch.softmax(model(input_ids).logits, dim=1)
        preds = torch.argmax(out, dim=1)

        
        tp += ((preds == 1) & (labels == 1)).sum().item()
        fp += ((preds == 1) & (labels == 0)).sum().item()
        tn += ((preds == 0) & (labels == 0)).sum().item()
        fn += ((preds == 0) & (labels == 1)).sum().item()

        acc = (tp + tn) / (tp + fp + tn + fn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1_score = 2 * precision * recall / (precision + recall)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


# XAI

In [ ]:
!pip install lime

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from lime.lime_text import LimeTextExplainer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Fairness/Bias detection/models/bert/checkpoint-800')


def predictor(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=1).numpy()
    return probabilities


explainer = LimeTextExplainer(class_names=['Non-biased','Biased']) 


